In [105]:
import pandas as pd
import numpy as np

In [106]:
data = pd.read_csv("log.csv")
data = data.rename(columns=lambda x: x.strip())
data = data.drop(["start_time", "end_time", "prompt"], axis=1)

In [107]:
data = data.loc[data["success"] == 1]
data = data.drop(["success", "task_class", "task_index"], axis=1)
data

,subject_id,task_round,mouse_distance,mouse_click,keyboard_press,duration
0,9,0,6282,30,28,72.235
1,9,0,9917,18,48,62.757
2,9,0,4208,13,1,36.350
3,9,0,9320,8,0,33.527
4,9,0,12359,14,14,46.989
...,...,...,...,...,...,...
372,13,1,0,0,53,18.611
373,13,1,0,0,32,18.680
375,13,1,0,0,56,18.704
376,13,1,1472,1,74,34.015


In [108]:
grouped = data.groupby(["subject_id", "task_round"], as_index=False)

In [122]:
group_data = grouped.sum()
print(group_data.columns)
# group_data = group_data.drop(["task_class", "tesk_index"], axis=1)  
group_data = round(group_data, 3)
group_data.to_csv("stat_origin.csv", index=False)

Index(['subject_id', 'task_round', 'mouse_distance', 'mouse_click',
       'keyboard_press', 'duration'],
      dtype='object')


In [110]:
overall_data = group_data.groupby("task_round").mean()
overall_data

,subject_id,mouse_distance,mouse_click,keyboard_press,duration
task_round,,,,,
0,9.0,161544.933333,326.333333,373.200000,763.424600
1,9.0,17693.066667,11.800000,1028.666667,428.398067


In [111]:
overall_std = group_data.groupby("task_round").std()
overall_std

,subject_id,mouse_distance,mouse_click,keyboard_press,duration
task_round,,,,,
0,4.472136,48662.337043,69.444394,102.154085,175.720044
1,4.472136,9087.926767,7.103319,246.873787,110.240513


In [112]:
overall_data.describe()

,subject_id,mouse_distance,mouse_click,keyboard_press,duration
count,2.0,2.000000,2.000000,2.000000,2.000000
mean,9.0,89619.000000,169.066667,700.933333,595.911333
std,0.0,101718.630406,222.408653,463.484925,236.899534
min,9.0,17693.066667,11.800000,373.200000,428.398067
25%,9.0,53656.033333,90.433333,537.066667,512.154700
50%,9.0,89619.000000,169.066667,700.933333,595.911333
75%,9.0,125581.966667,247.700000,864.800000,679.667967
max,9.0,161544.933333,326.333333,1028.666667,763.424600


In [113]:
import pandas as pd
import numpy as np
from scipy import stats

def summary(df):
    summary_df = pd.DataFrame(columns=['column_name', 'sample_size', 'mean', 'std', 'std_error', 'ci_lower', 'ci_upper', 'min', 'max'])
    dfs = []
    
    for column in df.columns:
        data = df[column]
        n = len(data)
        mean = np.mean(data)
        std = np.std(data)
        std_error = std / np.sqrt(n)
        ci = stats.t.interval(0.99, n-1, loc=mean, scale=std_error)
        ci_lower = ci[0]
        ci_upper = ci[1]

        minimum = np.min(data)
        maximum = np.max(data)
        
        dfs.append(pd.DataFrame({'column_name': column, 'sample_size': n, 'mean': mean, 
                                 'std': std, 'std_error': std_error, 'ci_lower': ci_lower, 
                                 'ci_upper': ci_upper, 'min': minimum, 'max': maximum}, index=[0]))
    
    summary_df = pd.concat(dfs, ignore_index=True)
    
    return summary_df

In [114]:
res = group_data
describe_0 = summary(res.loc[res["task_round"] == 0].drop(["subject_id", "task_round"], axis=1))

In [115]:
describe_1 =  summary(res.loc[res["task_round"] == 1].drop(["subject_id", "task_round"], axis=1))

In [116]:
describe_all = pd.concat([describe_0.assign(task_round=0), describe_1.assign(task_round=1)], ignore_index=True)

In [117]:
describe_all = describe_all.reindex(columns=['task_round', 'column_name', 'sample_size', 'mean', 'std', 'std_error', 'ci_lower', 'ci_upper', 'min', 'max'])

In [118]:
describe_all

,task_round,column_name,sample_size,mean,std,std_error,ci_lower,ci_upper,min,max
0,0,mouse_distance,15,161544.933333,47012.283963,12138.519524,125410.469686,197679.396980,98043.000,269443.000
1,0,mouse_click,15,326.333333,67.089658,17.322475,274.767049,377.899618,228.000,470.000
2,0,keyboard_press,15,373.200000,98.690222,25.481706,297.344969,449.055031,237.000,580.000
3,0,duration,15,763.424600,169.761690,43.832280,632.942796,893.906404,474.630,1143.378
4,1,mouse_distance,15,17693.066667,8779.771375,2266.927221,10944.780839,24441.352494,3804.000,43405.000
5,1,mouse_click,15,11.800000,6.862458,1.771879,6.525395,17.074605,2.000,26.000
6,1,keyboard_press,15,1028.666667,238.502737,61.581142,845.349292,1211.984041,753.000,1643.000
7,1,duration,15,428.398067,106.502454,27.498815,346.538418,510.257715,259.809,609.363


In [119]:
describe_all.to_csv("describe_all_stat.csv", index=False)